## Creación del Modelo y Entrenamiento

In [ ]:
# Clona tu repositorio
!git clone https://github.com/rarian1995/Navegacion_Autonoma_Proyecto_Final.git

In [ ]:
# Importar librerías
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

KeyboardInterrupt: 

In [ ]:
repo_path = '/content/Navegacion_Autonoma_Proyecto_Final'
csv_path = os.path.join(repo_path, 'final_log.csv')
image_dir = os.path.join(repo_path, 'dataset')

df = pd.read_csv(csv_path)
df['filepath'] = df['filename'].apply(lambda x: os.path.join(image_dir, x))

In [ ]:
# Cargar imágenes
def load_image(path, size=(64, 64)):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, size)
    return img / 255.0  # Normaliza a [0, 1]

X = np.array([load_image(p) for p in df['filepath']])
y = df['steering_angle'].values

In [ ]:
# Dividir en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = Sequential([
    Conv2D(24, (5, 5), strides=(2, 2), activation='relu', input_shape=X_train.shape[1:]),
    Conv2D(36, (5, 5), strides=(2, 2), activation='relu'),
    Conv2D(48, (5, 5), strides=(2, 2), activation='relu'),
    Flatten(),
    Dense(100, activation='relu'),
    Dense(50, activation='relu'),
    Dense(10, activation='relu'),
    Dense(1)
])

In [ ]:
model.compile(optimizer=Adam(1e-4), loss=MeanSquaredError())

In [ ]:
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=15,
    batch_size=32
)


In [ ]:
# Guardar el modelo
model.save('steering_model.h5')

In [ ]:
# Descargar el modelo a tu computadora
from google.colab import files
files.download('steering_model.h5')